<a href="https://colab.research.google.com/github/syh0397/Statistics_python/blob/main/12_%EB%8D%94%EB%AF%B8%EB%B3%80%EC%88%98(Dummy_Variable).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pingouin
!pip install --upgrade pingouin
!pip install scikit_posthocs
!pip install --upgrade patsy
!pip install patsy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
import pandas as pd 
import numpy as np 

import os
import sys
import warnings
warnings.filterwarnings('ignore')

import sklearn 
import missingno as msno
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.style.use('seaborn')
sns.set(font_scale=2.5)

import scipy 
from scipy import  stats
import statsmodels.api as sm
import pylab as py

import pingouin as pg
import scikit_posthocs

pd.options.display.float_format = '{:.3f}'.format

# 더미변수

- 회귀분석 - 연속형 독립변수 -> 연속형 종속변수에 미치는 영향을 알아봄

- 그러면 독립변수가 범주형일수는 없을까

- 그럴때 사용 하는게 더미변수 
- 원핫인코딩 처럼 사용한다.
  - 이진수로 해당하면 1 해당하지 않으면 0 
  - 피쳐의 갯수가 늘어난다.

In [7]:
hos = sns.load_dataset('taxis')
hos

,pickup,dropoff,passengers,distance,fare,tip,tolls,total,color,payment,pickup_zone,dropoff_zone,pickup_borough,dropoff_borough
0,2019-03-23 20:21:09,2019-03-23 20:27:24,1,1.600,7.000,2.150,0.000,12.950,yellow,credit card,Lenox Hill West,UN/Turtle Bay South,Manhattan,Manhattan
1,2019-03-04 16:11:55,2019-03-04 16:19:00,1,0.790,5.000,0.000,0.000,9.300,yellow,cash,Upper West Side South,Upper West Side South,Manhattan,Manhattan
2,2019-03-27 17:53:01,2019-03-27 18:00:25,1,1.370,7.500,2.360,0.000,14.160,yellow,credit card,Alphabet City,West Village,Manhattan,Manhattan
3,2019-03-10 01:23:59,2019-03-10 01:49:51,1,7.700,27.000,6.150,0.000,36.950,yellow,credit card,Hudson Sq,Yorkville West,Manhattan,Manhattan
4,2019-03-30 13:27:42,2019-03-30 13:37:14,3,2.160,9.000,1.100,0.000,13.400,yellow,credit card,Midtown East,Yorkville West,Manhattan,Manhattan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6428,2019-03-31 09:51:53,2019-03-31 09:55:27,1,0.750,4.500,1.060,0.000,6.360,green,credit card,East Harlem North,Central Harlem North,Manhattan,Manhattan
6429,2019-03-31 17:38:00,2019-03-31 18:34:23,1,18.740,58.000,0.000,0.000,58.800,green,credit card,Jamaica,East Concourse/Concourse Village,Queens,Bronx
6430,2019-03-23 22:55:18,2019-03-23 23:14:25,1,4.140,16.000,0.000,0.000,17.300,green,cash,Crown Heights North,Bushwick North,Brooklyn,Brooklyn
6431,2019-03-04 10:09:25,2019-03-04 10:14:29,1,1.120,6.000,0.000,0.000,6.800,green,credit card,East New York,East Flatbush/Remsen Village,Brooklyn,Brooklyn


In [15]:
import statsmodels.formula.api as smf
hos1 = pd.get_dummies(hos['color'], prefix = 'color', drop_first = False )
hos2 = pd.concat([hos, hos1], axis = 1)
print(hos1)

      color_green  color_yellow
0               0             1
1               0             1
2               0             1
3               0             1
4               0             1
...           ...           ...
6428            1             0
6429            1             0
6430            1             0
6431            1             0
6432            1             0

[6433 rows x 2 columns]


In [18]:
model = smf.ols(formula='tip~color_green+color_yellow', data=hos2).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    tip   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                  0.042
Method:                 Least Squares   F-statistic:                     141.3
Date:                Thu, 21 Apr 2022   Prob (F-statistic):           8.46e-61
Time:                        13:59:44   Log-Likelihood:                -14750.
No. Observations:                6433   AIC:                         2.951e+04
Df Residuals:                    6430   BIC:                         2.953e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept     1.395e+10   3.86e+12      0.004      0.997   -7.56e+12    7.58e+12
color_green  -1.395e+10   3.86e+12     -0.004      0.997   -7.58e+12    7.56e+12
color_yellow -1.395e+10   3.86e+12     -0.004      0.997   -7.58e+12    7.56e+12
==============================================================================
Omnibus:                     3930.781   Durbin-Watson:                   1.997
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            54969.425
Skew:                           2.698   Prob(JB):                         0.00
Kurtosis:                      16.265   Cond. No.                     2.99e+14
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.28e-25. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

- 팁과 color를 가지고 모형을 만들어 보았다. 
- 컬러가 2개라 2가지의 피쳐가 나온다. 맞으면 1 아님 0 


1. 모형의 설명력은 4% 밖에 안된다. 
2. 해석은 그 전의 11번 내용과 비슷한데 
3. 지금은 coef가 같아서 비슷해보이지만 다르다면 이렇게 해석한다.
  - 만약 color_green컬럼이 속한다면 기준집단 (intercept만 있는 집단) 보다 -1.395e+10  더 나아지고 
  - 거기에 color_yellow값이 추가된다면 1.395e+10  정도로 더 나아진다. (속하지 않았을때보다) 